In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import scipy
import sklearn
import math
%matplotlib inline
import seaborn as sns
from collections import Counter, defaultdict

# Text Library
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Splitting Data
from sklearn.model_selection import train_test_split

# Import libary for TFID Vectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# Import feature selection Libraries
from sklearn.feature_selection import SelectKBest, chi2, f_regression, mutual_info_classif

# Algorithm
from sklearn.naive_bayes import MultinomialNB

# Evaluation
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bimak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bimak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bimak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
dfTrain_pre = pd.read_csv('dfTrain_Pre.csv')
dfTest_pre = pd.read_csv('dfTest_Pre.csv')

In [3]:
#Extract TFIDF

# Convert dataset menjadi dataset dengan fitur2 tfidf
def conv_vector(dataset):
    X_data = dataset['tweet'].values
    y_data = dataset['label'].values

    # TFIDF Feature Matrix
    tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, lowercase = False, stop_words=None)
    # Fit transform to data train
    tfidf_data = tfidf_vectorizer.fit_transform(X_data)
    
    # get the first vector out (for the first document)
    first_vector_tfidfvectorizer=tfidf_data[0]

    # place tf-idf values in a pandas data frame
    df_fitur = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])

    df_fitur.reset_index(level=0, inplace=True)
    df_fitur.columns = ['feature', 'tfidf']
    
    return df_fitur

In [4]:
# tfidf feature from train_pre
tidf_train_pre = conv_vector(dfTrain_pre)

In [5]:
tidf_train_pre[tidf_train_pre['feature'].isnull() == True]

,feature,tfidf


In [6]:
fitur_pre = tidf_train_pre

In [7]:
len(fitur_pre.index)

5799

In [8]:
# Get each train and test values from dataset

# Train
X_train_pre = dfTrain_pre['tweet'].values
y_train_pre = dfTrain_pre['label'].values

# Test
X_test_pre = dfTest_pre['tweet'].values
y_test_pre = dfTest_pre['label'].values

In [9]:
# Get feature train and test for df_pre1
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, lowercase = False, stop_words=None)

# Fit transform to data train
tfidf_train_pre = tfidf_vectorizer.fit_transform(X_train_pre)
tfidf_test_pre = tfidf_vectorizer.transform(X_test_pre)

# Get feature matrix for train_pre1
fitur_train_pre = tfidf_train_pre.toarray()
fitur_tr2 = tfidf_vectorizer.get_feature_names()
train_pre_mtx = pd.DataFrame(data=fitur_train_pre,columns=fitur_tr2)

# Get feature matrix for test_pre1
fitur_test_pre = tfidf_test_pre.toarray()
fitur_te2 = tfidf_vectorizer.get_feature_names()
test_pre_mtx = pd.DataFrame(data=fitur_test_pre,columns=fitur_te2)

In [10]:
# Add labels into each matrix
train_pre_mtx['label_value'] = y_train_pre

In [11]:
def entropy(target_col):
    """
    Calculate the entropy of a dataset.
    The only parameter of this function is the target_col parameter which specifies the target column
    """
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

def InfoGain(data,split_attribute_name,target_name="class"):
    """
    Calculate the information gain of a dataset. This function takes three parameters:
    1. data = The dataset for whose feature the IG should be calculated
    2. split_attribute_name = the name of the feature for which the information gain should be calculated
    3. target_name = the name of the target feature. The default for this example is "class"
    """    
    #Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])
    
    ##Calculate the entropy of the dataset
    
    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    #Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

In [12]:
# Find IG for each df_pre2 feature
ig_pre_arr = []
listFitur_pre = list(fitur_pre.feature)
for i in range(len(listFitur_pre)):
    print('Fitur ke-',i)
    ig_pre_arr.append(InfoGain(train_pre_mtx, listFitur_pre[i], 'label_value'))

Fitur ke- 0
Fitur ke- 1
Fitur ke- 2
Fitur ke- 3
Fitur ke- 4
Fitur ke- 5
Fitur ke- 6
Fitur ke- 7
Fitur ke- 8
Fitur ke- 9
Fitur ke- 10
Fitur ke- 11
Fitur ke- 12
Fitur ke- 13
Fitur ke- 14
Fitur ke- 15
Fitur ke- 16
Fitur ke- 17
Fitur ke- 18
Fitur ke- 19
Fitur ke- 20
Fitur ke- 21
Fitur ke- 22
Fitur ke- 23
Fitur ke- 24
Fitur ke- 25
Fitur ke- 26
Fitur ke- 27
Fitur ke- 28
Fitur ke- 29
Fitur ke- 30
Fitur ke- 31
Fitur ke- 32
Fitur ke- 33
Fitur ke- 34
Fitur ke- 35
Fitur ke- 36
Fitur ke- 37
Fitur ke- 38
Fitur ke- 39
Fitur ke- 40
Fitur ke- 41
Fitur ke- 42
Fitur ke- 43
Fitur ke- 44
Fitur ke- 45
Fitur ke- 46
Fitur ke- 47
Fitur ke- 48
Fitur ke- 49
Fitur ke- 50
Fitur ke- 51
Fitur ke- 52
Fitur ke- 53
Fitur ke- 54
Fitur ke- 55
Fitur ke- 56
Fitur ke- 57
Fitur ke- 58
Fitur ke- 59
Fitur ke- 60
Fitur ke- 61
Fitur ke- 62
Fitur ke- 63
Fitur ke- 64
Fitur ke- 65
Fitur ke- 66
Fitur ke- 67
Fitur ke- 68
Fitur ke- 69
Fitur ke- 70
Fitur ke- 71
Fitur ke- 72
Fitur ke- 73
Fitur ke- 74
Fitur ke- 75
Fitur ke- 76
Fitur ke-

Fitur ke- 594
Fitur ke- 595
Fitur ke- 596
Fitur ke- 597
Fitur ke- 598
Fitur ke- 599
Fitur ke- 600
Fitur ke- 601
Fitur ke- 602
Fitur ke- 603
Fitur ke- 604
Fitur ke- 605
Fitur ke- 606
Fitur ke- 607
Fitur ke- 608
Fitur ke- 609
Fitur ke- 610
Fitur ke- 611
Fitur ke- 612
Fitur ke- 613
Fitur ke- 614
Fitur ke- 615
Fitur ke- 616
Fitur ke- 617
Fitur ke- 618
Fitur ke- 619
Fitur ke- 620
Fitur ke- 621
Fitur ke- 622
Fitur ke- 623
Fitur ke- 624
Fitur ke- 625
Fitur ke- 626
Fitur ke- 627
Fitur ke- 628
Fitur ke- 629
Fitur ke- 630
Fitur ke- 631
Fitur ke- 632
Fitur ke- 633
Fitur ke- 634
Fitur ke- 635
Fitur ke- 636
Fitur ke- 637
Fitur ke- 638
Fitur ke- 639
Fitur ke- 640
Fitur ke- 641
Fitur ke- 642
Fitur ke- 643
Fitur ke- 644
Fitur ke- 645
Fitur ke- 646
Fitur ke- 647
Fitur ke- 648
Fitur ke- 649
Fitur ke- 650
Fitur ke- 651
Fitur ke- 652
Fitur ke- 653
Fitur ke- 654
Fitur ke- 655
Fitur ke- 656
Fitur ke- 657
Fitur ke- 658
Fitur ke- 659
Fitur ke- 660
Fitur ke- 661
Fitur ke- 662
Fitur ke- 663
Fitur ke- 664
Fitur 

Fitur ke- 1168
Fitur ke- 1169
Fitur ke- 1170
Fitur ke- 1171
Fitur ke- 1172
Fitur ke- 1173
Fitur ke- 1174
Fitur ke- 1175
Fitur ke- 1176
Fitur ke- 1177
Fitur ke- 1178
Fitur ke- 1179
Fitur ke- 1180
Fitur ke- 1181
Fitur ke- 1182
Fitur ke- 1183
Fitur ke- 1184
Fitur ke- 1185
Fitur ke- 1186
Fitur ke- 1187
Fitur ke- 1188
Fitur ke- 1189
Fitur ke- 1190
Fitur ke- 1191
Fitur ke- 1192
Fitur ke- 1193
Fitur ke- 1194
Fitur ke- 1195
Fitur ke- 1196
Fitur ke- 1197
Fitur ke- 1198
Fitur ke- 1199
Fitur ke- 1200
Fitur ke- 1201
Fitur ke- 1202
Fitur ke- 1203
Fitur ke- 1204
Fitur ke- 1205
Fitur ke- 1206
Fitur ke- 1207
Fitur ke- 1208
Fitur ke- 1209
Fitur ke- 1210
Fitur ke- 1211
Fitur ke- 1212
Fitur ke- 1213
Fitur ke- 1214
Fitur ke- 1215
Fitur ke- 1216
Fitur ke- 1217
Fitur ke- 1218
Fitur ke- 1219
Fitur ke- 1220
Fitur ke- 1221
Fitur ke- 1222
Fitur ke- 1223
Fitur ke- 1224
Fitur ke- 1225
Fitur ke- 1226
Fitur ke- 1227
Fitur ke- 1228
Fitur ke- 1229
Fitur ke- 1230
Fitur ke- 1231
Fitur ke- 1232
Fitur ke- 1233
Fitur ke- 

Fitur ke- 1715
Fitur ke- 1716
Fitur ke- 1717
Fitur ke- 1718
Fitur ke- 1719
Fitur ke- 1720
Fitur ke- 1721
Fitur ke- 1722
Fitur ke- 1723
Fitur ke- 1724
Fitur ke- 1725
Fitur ke- 1726
Fitur ke- 1727
Fitur ke- 1728
Fitur ke- 1729
Fitur ke- 1730
Fitur ke- 1731
Fitur ke- 1732
Fitur ke- 1733
Fitur ke- 1734
Fitur ke- 1735
Fitur ke- 1736
Fitur ke- 1737
Fitur ke- 1738
Fitur ke- 1739
Fitur ke- 1740
Fitur ke- 1741
Fitur ke- 1742
Fitur ke- 1743
Fitur ke- 1744
Fitur ke- 1745
Fitur ke- 1746
Fitur ke- 1747
Fitur ke- 1748
Fitur ke- 1749
Fitur ke- 1750
Fitur ke- 1751
Fitur ke- 1752
Fitur ke- 1753
Fitur ke- 1754
Fitur ke- 1755
Fitur ke- 1756
Fitur ke- 1757
Fitur ke- 1758
Fitur ke- 1759
Fitur ke- 1760
Fitur ke- 1761
Fitur ke- 1762
Fitur ke- 1763
Fitur ke- 1764
Fitur ke- 1765
Fitur ke- 1766
Fitur ke- 1767
Fitur ke- 1768
Fitur ke- 1769
Fitur ke- 1770
Fitur ke- 1771
Fitur ke- 1772
Fitur ke- 1773
Fitur ke- 1774
Fitur ke- 1775
Fitur ke- 1776
Fitur ke- 1777
Fitur ke- 1778
Fitur ke- 1779
Fitur ke- 1780
Fitur ke- 

Fitur ke- 2262
Fitur ke- 2263
Fitur ke- 2264
Fitur ke- 2265
Fitur ke- 2266
Fitur ke- 2267
Fitur ke- 2268
Fitur ke- 2269
Fitur ke- 2270
Fitur ke- 2271
Fitur ke- 2272
Fitur ke- 2273
Fitur ke- 2274
Fitur ke- 2275
Fitur ke- 2276
Fitur ke- 2277
Fitur ke- 2278
Fitur ke- 2279
Fitur ke- 2280
Fitur ke- 2281
Fitur ke- 2282
Fitur ke- 2283
Fitur ke- 2284
Fitur ke- 2285
Fitur ke- 2286
Fitur ke- 2287
Fitur ke- 2288
Fitur ke- 2289
Fitur ke- 2290
Fitur ke- 2291
Fitur ke- 2292
Fitur ke- 2293
Fitur ke- 2294
Fitur ke- 2295
Fitur ke- 2296
Fitur ke- 2297
Fitur ke- 2298
Fitur ke- 2299
Fitur ke- 2300
Fitur ke- 2301
Fitur ke- 2302
Fitur ke- 2303
Fitur ke- 2304
Fitur ke- 2305
Fitur ke- 2306
Fitur ke- 2307
Fitur ke- 2308
Fitur ke- 2309
Fitur ke- 2310
Fitur ke- 2311
Fitur ke- 2312
Fitur ke- 2313
Fitur ke- 2314
Fitur ke- 2315
Fitur ke- 2316
Fitur ke- 2317
Fitur ke- 2318
Fitur ke- 2319
Fitur ke- 2320
Fitur ke- 2321
Fitur ke- 2322
Fitur ke- 2323
Fitur ke- 2324
Fitur ke- 2325
Fitur ke- 2326
Fitur ke- 2327
Fitur ke- 

...

Fitur ke- 3356
Fitur ke- 3357
Fitur ke- 3358
Fitur ke- 3359
Fitur ke- 3360
Fitur ke- 3361
Fitur ke- 3362
Fitur ke- 3363
Fitur ke- 3364
Fitur ke- 3365
Fitur ke- 3366
Fitur ke- 3367
Fitur ke- 3368
Fitur ke- 3369
Fitur ke- 3370
Fitur ke- 3371
Fitur ke- 3372
Fitur ke- 3373
Fitur ke- 3374
Fitur ke- 3375
Fitur ke- 3376
Fitur ke- 3377
Fitur ke- 3378
Fitur ke- 3379
Fitur ke- 3380
Fitur ke- 3381
Fitur ke- 3382
Fitur ke- 3383
Fitur ke- 3384
Fitur ke- 3385
Fitur ke- 3386
Fitur ke- 3387
Fitur ke- 3388
Fitur ke- 3389
Fitur ke- 3390
Fitur ke- 3391
Fitur ke- 3392
Fitur ke- 3393
Fitur ke- 3394
Fitur ke- 3395
Fitur ke- 3396
Fitur ke- 3397
Fitur ke- 3398
Fitur ke- 3399
Fitur ke- 3400
Fitur ke- 3401
Fitur ke- 3402
Fitur ke- 3403
Fitur ke- 3404
Fitur ke- 3405
Fitur ke- 3406
Fitur ke- 3407
Fitur ke- 3408
Fitur ke- 3409
Fitur ke- 3410
Fitur ke- 3411
Fitur ke- 3412
Fitur ke- 3413
Fitur ke- 3414
Fitur ke- 3415
Fitur ke- 3416
Fitur ke- 3417
Fitur ke- 3418
Fitur ke- 3419
Fitur ke- 3420
Fitur ke- 3421
Fitur ke- 

Fitur ke- 3903
Fitur ke- 3904
Fitur ke- 3905
Fitur ke- 3906
Fitur ke- 3907
Fitur ke- 3908
Fitur ke- 3909
Fitur ke- 3910
Fitur ke- 3911
Fitur ke- 3912
Fitur ke- 3913
Fitur ke- 3914
Fitur ke- 3915
Fitur ke- 3916
Fitur ke- 3917
Fitur ke- 3918
Fitur ke- 3919
Fitur ke- 3920
Fitur ke- 3921
Fitur ke- 3922
Fitur ke- 3923
Fitur ke- 3924
Fitur ke- 3925
Fitur ke- 3926
Fitur ke- 3927
Fitur ke- 3928
Fitur ke- 3929
Fitur ke- 3930
Fitur ke- 3931
Fitur ke- 3932
Fitur ke- 3933
Fitur ke- 3934
Fitur ke- 3935
Fitur ke- 3936
Fitur ke- 3937
Fitur ke- 3938
Fitur ke- 3939
Fitur ke- 3940
Fitur ke- 3941
Fitur ke- 3942
Fitur ke- 3943
Fitur ke- 3944
Fitur ke- 3945
Fitur ke- 3946
Fitur ke- 3947
Fitur ke- 3948
Fitur ke- 3949
Fitur ke- 3950
Fitur ke- 3951
Fitur ke- 3952
Fitur ke- 3953
Fitur ke- 3954
Fitur ke- 3955
Fitur ke- 3956
Fitur ke- 3957
Fitur ke- 3958
Fitur ke- 3959
Fitur ke- 3960
Fitur ke- 3961
Fitur ke- 3962
Fitur ke- 3963
Fitur ke- 3964
Fitur ke- 3965
Fitur ke- 3966
Fitur ke- 3967
Fitur ke- 3968
Fitur ke- 

Fitur ke- 4450
Fitur ke- 4451
Fitur ke- 4452
Fitur ke- 4453
Fitur ke- 4454
Fitur ke- 4455
Fitur ke- 4456
Fitur ke- 4457
Fitur ke- 4458
Fitur ke- 4459
Fitur ke- 4460
Fitur ke- 4461
Fitur ke- 4462
Fitur ke- 4463
Fitur ke- 4464
Fitur ke- 4465
Fitur ke- 4466
Fitur ke- 4467
Fitur ke- 4468
Fitur ke- 4469
Fitur ke- 4470
Fitur ke- 4471
Fitur ke- 4472
Fitur ke- 4473
Fitur ke- 4474
Fitur ke- 4475
Fitur ke- 4476
Fitur ke- 4477
Fitur ke- 4478
Fitur ke- 4479
Fitur ke- 4480
Fitur ke- 4481
Fitur ke- 4482
Fitur ke- 4483
Fitur ke- 4484
Fitur ke- 4485
Fitur ke- 4486
Fitur ke- 4487
Fitur ke- 4488
Fitur ke- 4489
Fitur ke- 4490
Fitur ke- 4491
Fitur ke- 4492
Fitur ke- 4493
Fitur ke- 4494
Fitur ke- 4495
Fitur ke- 4496
Fitur ke- 4497
Fitur ke- 4498
Fitur ke- 4499
Fitur ke- 4500
Fitur ke- 4501
Fitur ke- 4502
Fitur ke- 4503
Fitur ke- 4504
Fitur ke- 4505
Fitur ke- 4506
Fitur ke- 4507
Fitur ke- 4508
Fitur ke- 4509
Fitur ke- 4510
Fitur ke- 4511
Fitur ke- 4512
Fitur ke- 4513
Fitur ke- 4514
Fitur ke- 4515
Fitur ke- 

Fitur ke- 4997
Fitur ke- 4998
Fitur ke- 4999
Fitur ke- 5000
Fitur ke- 5001
Fitur ke- 5002
Fitur ke- 5003
Fitur ke- 5004
Fitur ke- 5005
Fitur ke- 5006
Fitur ke- 5007
Fitur ke- 5008
Fitur ke- 5009
Fitur ke- 5010
Fitur ke- 5011
Fitur ke- 5012
Fitur ke- 5013
Fitur ke- 5014
Fitur ke- 5015
Fitur ke- 5016
Fitur ke- 5017
Fitur ke- 5018
Fitur ke- 5019
Fitur ke- 5020
Fitur ke- 5021
Fitur ke- 5022
Fitur ke- 5023
Fitur ke- 5024
Fitur ke- 5025
Fitur ke- 5026
Fitur ke- 5027
Fitur ke- 5028
Fitur ke- 5029
Fitur ke- 5030
Fitur ke- 5031
Fitur ke- 5032
Fitur ke- 5033
Fitur ke- 5034
Fitur ke- 5035
Fitur ke- 5036
Fitur ke- 5037
Fitur ke- 5038
Fitur ke- 5039
Fitur ke- 5040
Fitur ke- 5041
Fitur ke- 5042
Fitur ke- 5043
Fitur ke- 5044
Fitur ke- 5045
Fitur ke- 5046
Fitur ke- 5047
Fitur ke- 5048
Fitur ke- 5049
Fitur ke- 5050
Fitur ke- 5051
Fitur ke- 5052
Fitur ke- 5053
Fitur ke- 5054
Fitur ke- 5055
Fitur ke- 5056
Fitur ke- 5057
Fitur ke- 5058
Fitur ke- 5059
Fitur ke- 5060
Fitur ke- 5061
Fitur ke- 5062
Fitur ke- 

Fitur ke- 5544
Fitur ke- 5545
Fitur ke- 5546
Fitur ke- 5547
Fitur ke- 5548
Fitur ke- 5549
Fitur ke- 5550
Fitur ke- 5551
Fitur ke- 5552
Fitur ke- 5553
Fitur ke- 5554
Fitur ke- 5555
Fitur ke- 5556
Fitur ke- 5557
Fitur ke- 5558
Fitur ke- 5559
Fitur ke- 5560
Fitur ke- 5561
Fitur ke- 5562
Fitur ke- 5563
Fitur ke- 5564
Fitur ke- 5565
Fitur ke- 5566
Fitur ke- 5567
Fitur ke- 5568
Fitur ke- 5569
Fitur ke- 5570
Fitur ke- 5571
Fitur ke- 5572
Fitur ke- 5573
Fitur ke- 5574
Fitur ke- 5575
Fitur ke- 5576
Fitur ke- 5577
Fitur ke- 5578
Fitur ke- 5579
Fitur ke- 5580
Fitur ke- 5581
Fitur ke- 5582
Fitur ke- 5583
Fitur ke- 5584
Fitur ke- 5585
Fitur ke- 5586
Fitur ke- 5587
Fitur ke- 5588
Fitur ke- 5589
Fitur ke- 5590
Fitur ke- 5591
Fitur ke- 5592
Fitur ke- 5593
Fitur ke- 5594
Fitur ke- 5595
Fitur ke- 5596
Fitur ke- 5597
Fitur ke- 5598
Fitur ke- 5599
Fitur ke- 5600
Fitur ke- 5601
Fitur ke- 5602
Fitur ke- 5603
Fitur ke- 5604
Fitur ke- 5605
Fitur ke- 5606
Fitur ke- 5607
Fitur ke- 5608
Fitur ke- 5609
Fitur ke- 

In [13]:
# Insert ig_value to fitur_prepre
fitur_pre['ig_value'] = ig_pre_arr

In [14]:
fitur_pre.to_csv('ig_pre.csv')

In [15]:
# Sort each fitur dataframe based on ig_value
fitur_pre_sorted = fitur_pre.sort_values(by='ig_value', ascending=False)
fitur_pre_sorted = list(fitur_pre_sorted.feature)

In [16]:
# Fungsi mengambil banyak top-k fitur yang harus diambil

def perc_fitur(train_vector):
    panjang = train_vector.shape[1]
    lenFitur = []
    for i in range (1,10):
      lenFitur.append(round((i/10)*panjang))
    lenFitur.reverse()
    
    return lenFitur

def generate_vec(train_data, test_data):
    X_train = train_data['tweet'].values
    y_train = train_data['label'].values

    X_test = test_data['tweet'].values
    y_test = test_data['label'].values
    
    tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, lowercase = False, stop_words=None)

    # Fit transform to data train
    tfidf_train = tfidf_vectorizer.fit_transform(X_train)
    tfidf_test = tfidf_vectorizer.transform(X_test)
    
    return [tfidf_train, tfidf_test]

In [17]:
# Generate feature vector for each dataset
vec_pre = generate_vec(dfTrain_pre, dfTest_pre)

In [18]:
# Ambil top-fitur setiap dataset dengan ukuran descending
lenFitur_pre = perc_fitur(vec_pre[0])

In [19]:
from sklearn import model_selection, svm

In [20]:
# Accuracy for dfpre
acc_ig_pre = []
for i in range(len(lenFitur_pre)):
    # Create subset feature based on top-n lenFitur
    subset_feature = fitur_pre_sorted[:lenFitur_pre[i]]
    
    # Create feature matrix for train and test
    features_train = scipy.sparse.csr_matrix(train_pre_mtx[subset_feature].values)
    features_test = scipy.sparse.csr_matrix(test_pre_mtx[subset_feature].values)
       
    # Classifier - Algorithm - SVM
    # fit the training dataset on the classifier
    SVM = svm.SVC(kernel = 'linear')
    SVM.fit(features_train, y_train_pre)

    # Use accuracy_score function to get the accuracy
#     print("SVM Training Accuracy Score -> ", SVM.score(features_train, y_train_prestem)*100)
    print("SVM Testing Accuracy Score -> ", SVM.score(features_test, y_test_pre)*100)

SVM Testing Accuracy Score ->  70.52238805970148
SVM Testing Accuracy Score ->  70.1492537313433
SVM Testing Accuracy Score ->  70.27363184079603
SVM Testing Accuracy Score ->  70.52238805970148
SVM Testing Accuracy Score ->  69.77611940298507
SVM Testing Accuracy Score ->  69.40298507462687
SVM Testing Accuracy Score ->  69.02985074626866
SVM Testing Accuracy Score ->  68.65671641791045
SVM Testing Accuracy Score ->  64.80099502487562


In [21]:
lenFitur_pre

[5219, 4639, 4059, 3479, 2900, 2320, 1740, 1160, 580]